<a href="https://colab.research.google.com/github/ygchoi-smwu/datamining/blob/main/dm21F_ch1_appendix_yongsangu_rental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 예제. API 크롤링 (계속)
# 공공데이터포털 OpenAPI "국토교통부_단독/다가구 전월세 자료"
### https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15058352
# 데이터 접근 위하여 API 접근 키 필요 --- 위 주소에서 신청 가능
API_KEY = "YOUR KEY HERE"


## URL 입력 (해당 API 매뉴얼 참고)
## 연립다세대 전월세 확정일자 조회시에는 지역코드, 거래년월, API 접근 키 필요
# 서비스 URL
urlService="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcRHRent"
# 지역코드 (용산구) 
lawd_cd = '11170'
# 거래년월 (yyyydd)
ym = '202108'


# URL 생성
url=urlService + "?LAWD_CD=" + lawd_cd+"&DEAL_YMD="+ym+"&serviceKey="+API_KEY


# 접근법
# 1. 웹브라우저 이용 (API 작동여부 테스트에 용이)
print(url)

# 2. urllib 이용
from urllib.request import urlopen
html = urlopen(url)
print( html.read().decode('UTF-8') )
# 이후 BeautifulSoup으로 파싱하면 됨


In [ ]:
# 예제. API 크롤링 (계속)
# 파싱 예제
# 출처: https://tariat.tistory.com/101

from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen

html = urlopen(url)
bs = BeautifulSoup(html.read(), 'lxml-xml')  
# html.parser보다 lxml-xml이 깔끔한 결과 제공, 한글 스트링 호환

itemlist = bs.find_all("item")

# print(itemlist[0])

df = pd.DataFrame()

for t in itemlist:
    build_y = t.find("건축년도").get_text()
    year = t.find("년").get_text()
    month = t.find("월").get_text()
    day = t.find("일").get_text()
    dong = t.find("법정동").get_text()
    bo_price = t.find("보증금액").get_text()
    mo_price = t.find("월세금액").get_text()
    apt_nm = t.find("연립다세대").get_text()
    size = t.find("전용면적").get_text()

    try:
        jibun = t.find("지번").get_text()
    except:
        jibun = ""

    ji_code = t.find("지역코드").get_text()
    floor = t.find("층").get_text()

    temp = pd.DataFrame(([[build_y,year,month,day,dong,bo_price,mo_price,apt_nm,size,jibun,ji_code,floor]]), 
                        columns=["build_y","year","month","day","dong","bo_price","mo_price","apt_nm","size","jibun","ji_code","floor"])

    df = pd.concat([df,temp])

df = df.reset_index(drop=True)

df

,build_y,year,month,day,dong,bo_price,mo_price,apt_nm,size,jibun,ji_code,floor
0,2021,2021,8,2,후암동,"27,000",0,"아인하우스(1,2동)",27.05,258-8,11170,3
1,1998,2021,8,4,후암동,"22,000",0,중앙그린빌라(27-7),49.98,27-7,11170,4
2,2006,2021,8,4,후암동,"25,000",0,훼미리빌,60.74,303-1,11170,5
3,2021,2021,8,5,후암동,"28,000",0,"아인하우스(1,2동)",29.94,258-8,11170,4
4,2021,2021,8,7,후암동,"30,000",0,용산 센트럴파크 평화1차,37.36,265-14,11170,3
...,...,...,...,...,...,...,...,...,...,...,...,...
188,2018,2021,8,18,동빙고동,"65,000",0,프리젠카운티,53.58,1-76,11170,1
189,2009,2021,8,18,서빙고동,"45,000",0,(198-1),61.19,198-1,11170,4
190,2017,2021,8,30,서빙고동,"30,500",0,용산파크빌라,41.07,4-140,11170,5
191,2004,2021,8,12,보광동,"18,000",0,(8-5),70.74,8-5,11170,4


In [ ]:
df.to_csv("용산구-연립다세대전월세-202108.csv", index=False)